### Convert TSG and Flow Meter Data to Correct Format for Inline Analysis

In [ ]:
TSG format is

lat=43 47.66737 N, lon=069 57.81890 W, hms=122029, dmy=040821, t1=18.254000, c1=4.118110, s=30.812400, sigma=22.007600

My TSG data is formatted as :
09/04/2021,22:57:51.709,        26.5469        36.0464      1539.56

    
My GPS data is:
09/04/2021,22:23:40.089,$GPZDA,222709.67,04,09,2021,00,00*67


#### Convert to proper format

In [65]:
import pandas as pd

tsg_original = pd.read_csv('data/SW21011R_RFP/TSG/TSG-_20210904-222337.Raw', sep='        |,|      ', header=None, names=['date', 'time', 'delete', 'T', 'S', 'unknown'])

gps_original = pd.read_csv('data/SW21011R_RFP/Navigation/SW-GGA_20210904-222337.Raw', header=None,
                           names=['date', 'time', 'gps', 'unknown', 'latdms', 'lat_sign', 'londms', 'lon_sign', '1', '2', '3', '4', '5', '6', '7'])

<ipython-input-65-12db4bb59c4b>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  tsg_original = pd.read_csv('data/SW21011R_RFP/TSG/TSG-_20210904-222337.Raw', sep='        |,|      ', header=None, names=['date', 'time', 'delete', 'T', 'S', 'unknown'])


In [66]:
tsg_original['datetime'] = pd.to_datetime(tsg_original['date'] + ' ' + tsg_original['time'])

In [67]:
tsg_original = tsg_original.set_index('datetime')

In [68]:
tsg_original['datetime'] = pd.to_datetime(tsg_original['date'] + ' ' + tsg_original['time'])

In [69]:
tsg_original

,date,time,delete,T,S,unknown,datetime
datetime,,,,,,,
2021-09-04 22:57:51.709,09/04/2021,22:57:51.709,NaN,26.5469,36.0464,1539.56,2021-09-04 22:57:51.709
2021-09-04 22:58:01.635,09/04/2021,22:58:01.635,NaN,26.5226,36.0374,1539.49,2021-09-04 22:58:01.635
2021-09-04 22:58:11.623,09/04/2021,22:58:11.623,NaN,26.4849,36.0074,1539.38,2021-09-04 22:58:11.623
2021-09-04 22:58:21.627,09/04/2021,22:58:21.627,NaN,26.4642,35.9757,1539.30,2021-09-04 22:58:21.627
2021-09-04 22:58:31.627,09/04/2021,22:58:31.627,NaN,26.4588,35.9478,1539.25,2021-09-04 22:58:31.627
...,...,...,...,...,...,...,...
2021-09-09 12:00:14.732,09/09/2021,12:00:14.732,NaN,27.4096,35.8314,1541.24,2021-09-09 12:00:14.732
2021-09-09 12:00:24.733,09/09/2021,12:00:24.733,NaN,27.4104,35.8320,1541.24,2021-09-09 12:00:24.733
2021-09-09 12:00:34.724,09/09/2021,12:00:34.724,NaN,27.4130,35.8316,1541.24,2021-09-09 12:00:34.724


In [70]:
gps_original['datetime'] = pd.to_datetime(gps_original['date'] + ' ' + gps_original['time'])

In [71]:
gps_original = gps_original.set_index('datetime')

In [72]:
indices = []
for i in range(len(tsg_original)):
    indices.append(gps_original.index.get_loc(tsg_original.iloc[i].datetime, method='nearest'))

In [73]:
gps_subset = gps_original.iloc[indices]

In [74]:
import numpy as np

def dms2dd(degrees, minutes, seconds, direction):
    dd = degrees + minutes/60 + seconds/(60*60);
    if direction == 'W' or direction == 'S':
        dd *= -1
    return dd;

gps_subset['lon'] = dms2dd(pd.to_numeric(gps_subset['londms'].astype(str).str[:2]).astype(float), pd.to_numeric(gps_subset['londms'].astype(str).str[2:]),np.array(0), 'W')
gps_subset['lat'] = dms2dd(pd.to_numeric(gps_subset['latdms'].astype(str).str[:2]).astype(float), pd.to_numeric(gps_subset['latdms'].astype(str).str[2:]),np.array(0), 'E')

<ipython-input-74-9342abf7a88e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gps_subset['lon'] = dms2dd(pd.to_numeric(gps_subset['londms'].astype(str).str[:2]).astype(float), pd.to_numeric(gps_subset['londms'].astype(str).str[2:]),np.array(0), 'W')
<ipython-input-74-9342abf7a88e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gps_subset['lat'] = dms2dd(pd.to_numeric(gps_subset['latdms'].astype(str).str[:2]).astype(float), pd.to_numeric(gps_subset['latdms'].astype(str).str[2:]),np.array(0), 'E')


In [75]:
gps_subset.head()

,date,time,gps,unknown,latdms,lat_sign,londms,lon_sign,1,2,3,4,5,6,7,lon,lat
datetime,,,,,,,,,,,,,,,,,
2021-09-04 22:57:51.969,09/04/2021,22:57:51.969,$GPGGA,230114.00,3438.1081,N,7639.5340,W,2,28,0.5,-27.0,M,-36.9,M*73,-76.658900,34.635135
2021-09-04 22:58:01.949,09/04/2021,22:58:01.949,$GPGGA,230123.68,3438.0566,N,7639.5045,W,2,28,0.5,-26.4,M,-36.9,M*47,-76.658408,34.634277
2021-09-04 22:58:12.102,09/04/2021,22:58:12.102,$GPGGA,230133.68,3438.0046,N,7639.4760,W,2,28,0.5,-26.6,M,-36.9,M*42,-76.657933,34.633410
2021-09-04 22:58:21.706,09/04/2021,22:58:21.706,$GPGGA,230143.69,3437.9533,N,7639.4463,W,2,27,0.5,-27.2,M,-36.9,M*4F,-76.657438,34.632555
2021-09-04 22:58:31.002,09/04/2021,22:58:31.002,$GPGGA,230152.69,3437.9070,N,7639.4200,W,2,27,0.5,-27.0,M,-36.9,M*52,-76.657000,34.631783


In [76]:
tsg_original['latdms'] = gps_subset['latdms'].values
tsg_original['londms'] = gps_subset['londms'].values

In [ ]:
lat=43 47.66737 N, lon=069 57.81890 W, hms=122029, dmy=040821, t1=18.254000, c1=4.118110, s=30.812400, sigma=22.007600

In [105]:
tsg_original.iloc[1].datetime.strftime('%H%M%S') # %d/%m/%y

'225801'

In [113]:
row.datetime.strftime('%d%m%y')

'040921'

In [115]:
row['T']

26.5469

In [122]:
f = open('tsg_20210904.txt', 'w')
i=0
for index, row in tsg_original.iterrows():
    if i > len(tsg_original):
        break
    else:
        f.write('lat='+str(row['latdms'])[:2]+' '+str(row['latdms'])[2:]+' N, ')
        f.write('lon=0'+str(row['latdms'])[:2]+' '+str(row['latdms'])[2:]+' W, ')
        f.write('hms='+row.datetime.strftime('%H%M%S')+', ')
        f.write('dmy='+row.datetime.strftime('%d%m%y')+', ')
        f.write('t1='+str(row['T'])+', ')
        f.write('c1=0, ')
        f.write('s='+str(row['S'])+', ')
        f.write('sigma=0\n')
        i+=1
f.close()

### Converting flow control to proper format

proper format is 
2021-08-04 19:00:01 UTC	1	0110	0.00	43.48	0.000	2.469
        
my format is
09/04/2021,22:23:38.407,$FLOW,0,mL/s

In [127]:
flow_df = pd.read_csv('data/SW21011R_RFP/TSG/FlowMeter-_20210904-222337.Raw', header=None, names=['date', 'time', 'flowname', 'flow', 'units'])

In [128]:
flow_df

,date,time,flowname,flow,units
0,09/04/2021,22:23:38.407,$FLOW,0.0,mL/s
1,09/04/2021,22:23:39.415,$FLOW,0.0,mL/s
2,09/04/2021,22:23:40.418,$FLOW,0.0,mL/s
3,09/04/2021,22:23:41.431,$FLOW,0.0,mL/s
4,09/04/2021,22:23:42.433,$FLOW,0.0,mL/s
...,...,...,...,...,...
389779,09/09/2021,12:01:10.306,$FLOW,0.0,mL/s
389780,09/09/2021,12:01:11.313,$FLOW,0.0,mL/s
389781,09/09/2021,12:01:12.318,$FLOW,0.0,mL/s
389782,09/09/2021,12:01:13.324,$FLOW,0.0,mL/s


In [130]:
flow_df['datetime'] = pd.to_datetime(flow_df.date + ' ' + flow_df.time)

In [131]:
flow_df.head()

,date,time,flowname,flow,units,datetime
0,09/04/2021,22:23:38.407,$FLOW,0.0,mL/s,2021-09-04 22:23:38.407
1,09/04/2021,22:23:39.415,$FLOW,0.0,mL/s,2021-09-04 22:23:39.415
2,09/04/2021,22:23:40.418,$FLOW,0.0,mL/s,2021-09-04 22:23:40.418
3,09/04/2021,22:23:41.431,$FLOW,0.0,mL/s,2021-09-04 22:23:41.431
4,09/04/2021,22:23:42.433,$FLOW,0.0,mL/s,2021-09-04 22:23:42.433


In [ ]:
2021-08-04 19:00:01 UTC	1	0110	0.00	43.48	0.000	2.469
2021-09-04 22:23:38 UTC	1	0110	0.00	50.0	0.00	0.0

In [132]:
flow_df.iloc[10].datetime.strftime('%H:%M:%S')

'22:23:48'

In [144]:
flow_df.iloc[10].datetime.strftime('%Y-%m-%d')

'2021-09-04'

1.05

In [151]:
f = open('Flow_20210904.txt', 'w')
i=0
for index, row in flow_df.iterrows():
    if i > len(flow_df):
        break
    else:
        f.write(row.datetime.strftime('%Y-%m-%d')+' '+row.datetime.strftime('%H:%M:%S')+' ')
        f.write('UTC\t1\t')
        f.write('0110\t0.00\t')
        f.write(str(row['flow']*60/1000)+'\t')
        f.write('0.00\t')
        f.write('0.00\n')
        
        i+=1
f.close()